In [2]:
import os
import os.path
root = os.getcwd()
class basic_block:
   def __init__(self, function, name):
      self.function = function
      self.name = name
      self.pre_block = 0.0
      self.post_block = 0.0
      self.input = 0.0
      self.output = 0.0
      self.external_call = 0
      self.inital = 0
      self.ret = 0
      self.memory = 0.0
  
        
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np     

block_names = []
with open(os.path.join(root,"motivationalResults")) as motivation_file:
  lines = motivation_file.readlines()
  for line in lines:
    tempory = line.split(" ")
    name = tempory[0]+" "+tempory[1]
    if float(tempory[3])>0:
      block_names.append(name)
  


block_informations = []   
with open(os.path.join(root,"basicBlockTrace")) as trace_file:
  lines = trace_file.readlines()
  cur_function = ""
  max_input = 1
  max_output = 1
  block_number = 0
  
  tempory_informations = []
  for line in lines[1:]:
    tempory = line.split(" ")
    
    if cur_function == "" or cur_function != tempory[0]:
      if cur_function != tempory[0]:
        for tempory_information in tempory_informations:
          tempory_information.input = tempory_information.input/max_input
          tempory_information.output = tempory_information.output/max_output
          tempory_information.pre_block = tempory_information.pre_block/block_number
          tempory_information.post_block = tempory_information.post_block/block_number
          block_informations.append(tempory_information)
        max_input = 1
        max_output = 1
        block_number = 0
        tempory_informations = []
      cur_function = tempory[0]


    block_number += 1
    block_information = basic_block(cur_function,tempory[1])
    block_information.pre_block = float(tempory[2])
    block_information.post_block = float(tempory[3])
    block_information.input = float(tempory[4])
    block_information.output = float(tempory[5])
    max_input = max(max_input,block_information.input)
    max_output = max(max_output,block_information.output)
    block_information.external_call = int(tempory[6])
    block_information.inital = int(tempory[7])
    block_information.ret = int(tempory[8])
    block_information.memory = float(tempory[9])/float(tempory[10])
    name = tempory[0]+" "+tempory[1]
    if name in block_names:
      tempory_informations.append(block_information)
  
  for tempory_information in tempory_informations:
    tempory_information.input = tempory_information.input/max_input
    tempory_information.output = tempory_information.output/max_output
    tempory_information.pre_block = tempory_information.pre_block/block_number
    tempory_information.post_block = tempory_information.post_block/block_number
    block_informations.append(tempory_information)

In [3]:
block_data = np.empty([len(block_informations), 8], dtype = float)

for i in range(len(block_informations)):
  block_data[i] = [block_informations[i].pre_block,block_informations[i].post_block,block_informations[i].input,
                    block_informations[i].output,block_informations[i].external_call,block_informations[i].inital,
                    block_informations[i].ret,block_informations[i].memory]
  

block_labels = np.empty([len(block_informations), 1], dtype =int)
i = 0
with open(os.path.join(root,"experimentResults")) as sensivity_file:
  lines = sensivity_file.readlines()
  for line in lines:
    tempory = line.split(" ")
    name = tempory[0]+" "+tempory[1]
    if name not in block_names:
      continue
    if "N" in tempory[2]:
      block_labels[i] = int(0)
    else:
      block_labels[i] = int(1)
    i = i+1  
    
print(len(block_informations))
print(len(block_labels))
X_train, X_test, y_train, y_test = train_test_split(block_data,block_labels, test_size=0.33,shuffle=True)

23
23


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
import datetime

def subtime(date1, date2):
    date1 = datetime.datetime.strptime(date1, "%Y-%m-%d %H:%M:%S")
    date2 = datetime.datetime.strptime(date2, "%Y-%m-%d %H:%M:%S")
    return date2 - date1


model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_dim=X_train.shape[1]),
    keras.layers.Dense(2, activation='sigmoid')
])

y_binary_train= to_categorical(y_train)
y_binary_test = to_categorical(y_test)

sgd = SGD(0.005)
model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

startdate = datetime.datetime.now() # 获取当前时间
startdate = startdate.strftime("%Y-%m-%d %H:%M:%S") # 当前时间转换为指定字符串格式
H = model.fit(X_train, y_binary_train,epochs = 500)

enddate = datetime.datetime.now() # 获取当前时间
enddate = enddate.strftime("%Y-%m-%d %H:%M:%S") # 当前时间转换为指定字符串格式

E = model.evaluate(X_test, y_binary_test)

print('TrainTime ',subtime(startdate,enddate))

Epoch 1/500
15/15 [==============================] - 0s 10ms/sample - loss: 0.7395 - acc: 0.1333
Epoch 2/500
15/15 [==============================] - 0s 215us/sample - loss: 0.7386 - acc: 0.1333
Epoch 3/500
15/15 [==============================] - 0s 162us/sample - loss: 0.7376 - acc: 0.1667
Epoch 4/500
15/15 [==============================] - 0s 171us/sample - loss: 0.7366 - acc: 0.1667
Epoch 5/500
15/15 [==============================] - 0s 205us/sample - loss: 0.7357 - acc: 0.2333
Epoch 6/500
15/15 [==============================] - 0s 168us/sample - loss: 0.7347 - acc: 0.2333
Epoch 7/500
15/15 [==============================] - 0s 150us/sample - loss: 0.7337 - acc: 0.2333
Epoch 8/500
15/15 [==============================] - 0s 175us/sample - loss: 0.7328 - acc: 0.2667
Epoch 9/500
15/15 [==============================] - 0s 170us/sample - loss: 0.7319 - acc: 0.3000
Epoch 10/500
15/15 [==============================] - 0s 168us/sample - loss: 0.7309 - acc: 0.3333
Epoch 11/500
15/15 [

Epoch 84/500
15/15 [==============================] - 0s 206us/sample - loss: 0.6761 - acc: 0.7333
Epoch 85/500
15/15 [==============================] - 0s 141us/sample - loss: 0.6755 - acc: 0.7333
Epoch 86/500
15/15 [==============================] - 0s 144us/sample - loss: 0.6749 - acc: 0.7333
Epoch 87/500
15/15 [==============================] - 0s 201us/sample - loss: 0.6744 - acc: 0.7333
Epoch 88/500
15/15 [==============================] - 0s 153us/sample - loss: 0.6738 - acc: 0.7333
Epoch 89/500
15/15 [==============================] - 0s 216us/sample - loss: 0.6733 - acc: 0.7333
Epoch 90/500
15/15 [==============================] - 0s 164us/sample - loss: 0.6727 - acc: 0.7333
Epoch 91/500
15/15 [==============================] - 0s 164us/sample - loss: 0.6722 - acc: 0.7333
Epoch 92/500
15/15 [==============================] - 0s 149us/sample - loss: 0.6717 - acc: 0.7333
Epoch 93/500
15/15 [==============================] - 0s 166us/sample - loss: 0.6711 - acc: 0.7333
Epoch 94/5

15/15 [==============================] - 0s 149us/sample - loss: 0.6382 - acc: 0.7667
Epoch 167/500
15/15 [==============================] - 0s 152us/sample - loss: 0.6378 - acc: 0.8000
Epoch 168/500
15/15 [==============================] - 0s 157us/sample - loss: 0.6375 - acc: 0.8000
Epoch 169/500
15/15 [==============================] - 0s 161us/sample - loss: 0.6371 - acc: 0.8000
Epoch 170/500
15/15 [==============================] - 0s 153us/sample - loss: 0.6367 - acc: 0.8000
Epoch 171/500
15/15 [==============================] - 0s 143us/sample - loss: 0.6363 - acc: 0.8000
Epoch 172/500
15/15 [==============================] - 0s 147us/sample - loss: 0.6360 - acc: 0.8000
Epoch 173/500
15/15 [==============================] - 0s 147us/sample - loss: 0.6356 - acc: 0.8000
Epoch 174/500
15/15 [==============================] - 0s 157us/sample - loss: 0.6352 - acc: 0.8000
Epoch 175/500
15/15 [==============================] - 0s 157us/sample - loss: 0.6349 - acc: 0.8000
Epoch 176/500


Epoch 249/500
15/15 [==============================] - 0s 147us/sample - loss: 0.6111 - acc: 0.8000
Epoch 250/500
15/15 [==============================] - 0s 137us/sample - loss: 0.6108 - acc: 0.8000
Epoch 251/500
15/15 [==============================] - 0s 142us/sample - loss: 0.6105 - acc: 0.8000
Epoch 252/500
15/15 [==============================] - 0s 157us/sample - loss: 0.6102 - acc: 0.8000
Epoch 253/500
15/15 [==============================] - 0s 152us/sample - loss: 0.6100 - acc: 0.8000
Epoch 254/500
15/15 [==============================] - 0s 150us/sample - loss: 0.6097 - acc: 0.8000
Epoch 255/500
15/15 [==============================] - 0s 141us/sample - loss: 0.6094 - acc: 0.8000
Epoch 256/500
15/15 [==============================] - 0s 159us/sample - loss: 0.6091 - acc: 0.8000
Epoch 257/500
15/15 [==============================] - 0s 138us/sample - loss: 0.6089 - acc: 0.8000
Epoch 258/500
15/15 [==============================] - 0s 156us/sample - loss: 0.6086 - acc: 0.8000


Epoch 331/500
15/15 [==============================] - 0s 144us/sample - loss: 0.5906 - acc: 0.8000
Epoch 332/500
15/15 [==============================] - 0s 137us/sample - loss: 0.5904 - acc: 0.8000
Epoch 333/500
15/15 [==============================] - 0s 153us/sample - loss: 0.5902 - acc: 0.8000
Epoch 334/500
15/15 [==============================] - 0s 154us/sample - loss: 0.5900 - acc: 0.8000
Epoch 335/500
15/15 [==============================] - 0s 147us/sample - loss: 0.5898 - acc: 0.8000
Epoch 336/500
15/15 [==============================] - 0s 143us/sample - loss: 0.5896 - acc: 0.8000
Epoch 337/500
15/15 [==============================] - 0s 150us/sample - loss: 0.5894 - acc: 0.8000
Epoch 338/500
15/15 [==============================] - 0s 137us/sample - loss: 0.5891 - acc: 0.8000
Epoch 339/500
15/15 [==============================] - 0s 154us/sample - loss: 0.5889 - acc: 0.8000
Epoch 340/500
15/15 [==============================] - 0s 155us/sample - loss: 0.5887 - acc: 0.8000


Epoch 413/500
15/15 [==============================] - 0s 135us/sample - loss: 0.5751 - acc: 0.8000
Epoch 414/500
15/15 [==============================] - 0s 139us/sample - loss: 0.5749 - acc: 0.8000
Epoch 415/500
15/15 [==============================] - 0s 137us/sample - loss: 0.5748 - acc: 0.8000
Epoch 416/500
15/15 [==============================] - 0s 135us/sample - loss: 0.5746 - acc: 0.8000
Epoch 417/500
15/15 [==============================] - 0s 153us/sample - loss: 0.5744 - acc: 0.8000
Epoch 418/500
15/15 [==============================] - 0s 135us/sample - loss: 0.5743 - acc: 0.8000
Epoch 419/500
15/15 [==============================] - 0s 138us/sample - loss: 0.5741 - acc: 0.8000
Epoch 420/500
15/15 [==============================] - 0s 138us/sample - loss: 0.5739 - acc: 0.8000
Epoch 421/500
15/15 [==============================] - 0s 141us/sample - loss: 0.5738 - acc: 0.8000
Epoch 422/500
15/15 [==============================] - 0s 135us/sample - loss: 0.5736 - acc: 0.8000


Epoch 495/500
15/15 [==============================] - 0s 138us/sample - loss: 0.5626 - acc: 0.8000
Epoch 496/500
15/15 [==============================] - 0s 155us/sample - loss: 0.5625 - acc: 0.8000
Epoch 497/500
15/15 [==============================] - 0s 147us/sample - loss: 0.5623 - acc: 0.8000
Epoch 498/500
15/15 [==============================] - 0s 155us/sample - loss: 0.5622 - acc: 0.8000
Epoch 499/500
15/15 [==============================] - 0s 142us/sample - loss: 0.5621 - acc: 0.8000
Epoch 500/500
8/8 [==============================] - 0s 8ms/sample - loss: 0.4164 - acc: 1.0000
TrainTime  0:00:03
TestTime  0:00:00


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical

model = Sequential()
model.add(Dense(2,input_shape = (X_train.shape[1],),activation = 'sigmoid'))

y_binary_train= to_categorical(y_train)
y_binary_test = to_categorical(y_test)

sgd = SGD(0.005)
model.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics=['accuracy'])

H = model.fit(X_train, y_binary_train,epochs = 1000)
E = model.evaluate(X_test, y_binary_test)

Epoch 1/1000
11/11 [==============================] - 0s 12ms/sample - loss: 0.6336 - acc: 0.6364
Epoch 2/1000
11/11 [==============================] - 0s 191us/sample - loss: 0.6330 - acc: 0.6364
Epoch 3/1000
11/11 [==============================] - 0s 192us/sample - loss: 0.6325 - acc: 0.6364
Epoch 4/1000
11/11 [==============================] - 0s 186us/sample - loss: 0.6319 - acc: 0.6364
Epoch 5/1000
11/11 [==============================] - 0s 182us/sample - loss: 0.6314 - acc: 0.6364
Epoch 6/1000
11/11 [==============================] - 0s 184us/sample - loss: 0.6308 - acc: 0.6364
Epoch 7/1000
11/11 [==============================] - 0s 189us/sample - loss: 0.6303 - acc: 0.6364
Epoch 8/1000
11/11 [==============================] - 0s 186us/sample - loss: 0.6298 - acc: 0.6364
Epoch 9/1000
11/11 [==============================] - 0s 187us/sample - loss: 0.6292 - acc: 0.6364
Epoch 10/1000
11/11 [==============================] - 0s 178us/sample - loss: 0.6287 - acc: 0.6364
Epoch 11/1

11/11 [==============================] - 0s 180us/sample - loss: 0.5920 - acc: 0.7727
Epoch 84/1000
11/11 [==============================] - 0s 180us/sample - loss: 0.5915 - acc: 0.7727
Epoch 85/1000
11/11 [==============================] - 0s 181us/sample - loss: 0.5911 - acc: 0.7727
Epoch 86/1000
11/11 [==============================] - 0s 180us/sample - loss: 0.5906 - acc: 0.7727
Epoch 87/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.5901 - acc: 0.7727
Epoch 88/1000
11/11 [==============================] - 0s 175us/sample - loss: 0.5897 - acc: 0.7727
Epoch 89/1000
11/11 [==============================] - 0s 179us/sample - loss: 0.5892 - acc: 0.7727
Epoch 90/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.5887 - acc: 0.7727
Epoch 91/1000
11/11 [==============================] - 0s 183us/sample - loss: 0.5883 - acc: 0.8182
Epoch 92/1000
11/11 [==============================] - 0s 189us/sample - loss: 0.5878 - acc: 0.8182
Epoch 93/1000


Epoch 165/1000
11/11 [==============================] - 0s 184us/sample - loss: 0.5563 - acc: 0.8636
Epoch 166/1000
11/11 [==============================] - 0s 196us/sample - loss: 0.5559 - acc: 0.8636
Epoch 167/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.5555 - acc: 0.8636
Epoch 168/1000
11/11 [==============================] - 0s 202us/sample - loss: 0.5551 - acc: 0.8636
Epoch 169/1000
11/11 [==============================] - 0s 181us/sample - loss: 0.5547 - acc: 0.8636
Epoch 170/1000
11/11 [==============================] - 0s 182us/sample - loss: 0.5543 - acc: 0.8636
Epoch 171/1000
11/11 [==============================] - 0s 196us/sample - loss: 0.5539 - acc: 0.8636
Epoch 172/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.5535 - acc: 0.8636
Epoch 173/1000
11/11 [==============================] - 0s 186us/sample - loss: 0.5531 - acc: 0.8636
Epoch 174/1000
11/11 [==============================] - 0s 187us/sample - loss: 0.5527 - ac

11/11 [==============================] - 0s 178us/sample - loss: 0.5258 - acc: 0.9091
Epoch 247/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.5255 - acc: 0.9091
Epoch 248/1000
11/11 [==============================] - 0s 188us/sample - loss: 0.5251 - acc: 0.9091
Epoch 249/1000
11/11 [==============================] - 0s 201us/sample - loss: 0.5248 - acc: 0.9091
Epoch 250/1000
11/11 [==============================] - 0s 182us/sample - loss: 0.5244 - acc: 0.9091
Epoch 251/1000
11/11 [==============================] - 0s 175us/sample - loss: 0.5241 - acc: 0.9091
Epoch 252/1000
11/11 [==============================] - 0s 175us/sample - loss: 0.5237 - acc: 0.9091
Epoch 253/1000
11/11 [==============================] - 0s 208us/sample - loss: 0.5234 - acc: 0.9091
Epoch 254/1000
11/11 [==============================] - 0s 179us/sample - loss: 0.5230 - acc: 0.9091
Epoch 255/1000
11/11 [==============================] - 0s 174us/sample - loss: 0.5227 - acc: 0.9091
Epoch

Epoch 328/1000
11/11 [==============================] - 0s 181us/sample - loss: 0.4991 - acc: 0.9091
Epoch 329/1000
11/11 [==============================] - 0s 194us/sample - loss: 0.4988 - acc: 0.9091
Epoch 330/1000
11/11 [==============================] - 0s 183us/sample - loss: 0.4985 - acc: 0.9091
Epoch 331/1000
11/11 [==============================] - 0s 180us/sample - loss: 0.4982 - acc: 0.9091
Epoch 332/1000
11/11 [==============================] - 0s 174us/sample - loss: 0.4979 - acc: 0.9091
Epoch 333/1000
11/11 [==============================] - 0s 173us/sample - loss: 0.4976 - acc: 0.9091
Epoch 334/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.4973 - acc: 0.9091
Epoch 335/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.4970 - acc: 0.9091
Epoch 336/1000
11/11 [==============================] - 0s 194us/sample - loss: 0.4967 - acc: 0.9091
Epoch 337/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.4964 - ac

...

Epoch 654/1000
11/11 [==============================] - 0s 185us/sample - loss: 0.4228 - acc: 0.9091
Epoch 655/1000
11/11 [==============================] - 0s 188us/sample - loss: 0.4226 - acc: 0.9091
Epoch 656/1000
11/11 [==============================] - 0s 191us/sample - loss: 0.4224 - acc: 0.9091
Epoch 657/1000
11/11 [==============================] - 0s 195us/sample - loss: 0.4223 - acc: 0.9091
Epoch 658/1000
11/11 [==============================] - 0s 202us/sample - loss: 0.4221 - acc: 0.9091
Epoch 659/1000
11/11 [==============================] - 0s 208us/sample - loss: 0.4219 - acc: 0.9091
Epoch 660/1000
11/11 [==============================] - 0s 180us/sample - loss: 0.4217 - acc: 0.9091
Epoch 661/1000
11/11 [==============================] - 0s 180us/sample - loss: 0.4215 - acc: 0.9091
Epoch 662/1000
11/11 [==============================] - 0s 184us/sample - loss: 0.4214 - acc: 0.9091
Epoch 663/1000
11/11 [==============================] - 0s 200us/sample - loss: 0.4212 - ac

11/11 [==============================] - 0s 181us/sample - loss: 0.4092 - acc: 0.9091
Epoch 736/1000
11/11 [==============================] - 0s 184us/sample - loss: 0.4090 - acc: 0.9091
Epoch 737/1000
11/11 [==============================] - 0s 193us/sample - loss: 0.4088 - acc: 0.9091
Epoch 738/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.4087 - acc: 0.9091
Epoch 739/1000
11/11 [==============================] - 0s 184us/sample - loss: 0.4085 - acc: 0.9091
Epoch 740/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.4084 - acc: 0.9091
Epoch 741/1000
11/11 [==============================] - 0s 172us/sample - loss: 0.4082 - acc: 0.9091
Epoch 742/1000
11/11 [==============================] - 0s 203us/sample - loss: 0.4081 - acc: 0.9091
Epoch 743/1000
11/11 [==============================] - 0s 175us/sample - loss: 0.4079 - acc: 0.9091
Epoch 744/1000
11/11 [==============================] - 0s 201us/sample - loss: 0.4077 - acc: 0.9091
Epoch

Epoch 817/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.3969 - acc: 0.9091
Epoch 818/1000
11/11 [==============================] - 0s 181us/sample - loss: 0.3968 - acc: 0.9091
Epoch 819/1000
11/11 [==============================] - 0s 191us/sample - loss: 0.3967 - acc: 0.9091
Epoch 820/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.3965 - acc: 0.9091
Epoch 821/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.3964 - acc: 0.9091
Epoch 822/1000
11/11 [==============================] - 0s 178us/sample - loss: 0.3962 - acc: 0.9091
Epoch 823/1000
11/11 [==============================] - 0s 203us/sample - loss: 0.3961 - acc: 0.9091
Epoch 824/1000
11/11 [==============================] - 0s 212us/sample - loss: 0.3960 - acc: 0.9091
Epoch 825/1000
11/11 [==============================] - 0s 211us/sample - loss: 0.3958 - acc: 0.9091
Epoch 826/1000
11/11 [==============================] - 0s 178us/sample - loss: 0.3957 - ac

11/11 [==============================] - 0s 183us/sample - loss: 0.3862 - acc: 0.9091
Epoch 899/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.3861 - acc: 0.9091
Epoch 900/1000
11/11 [==============================] - 0s 186us/sample - loss: 0.3859 - acc: 0.9091
Epoch 901/1000
11/11 [==============================] - 0s 174us/sample - loss: 0.3858 - acc: 0.9091
Epoch 902/1000
11/11 [==============================] - 0s 182us/sample - loss: 0.3857 - acc: 0.9091
Epoch 903/1000
11/11 [==============================] - 0s 194us/sample - loss: 0.3856 - acc: 0.9091
Epoch 904/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.3854 - acc: 0.9091
Epoch 905/1000
11/11 [==============================] - 0s 194us/sample - loss: 0.3853 - acc: 0.9091
Epoch 906/1000
11/11 [==============================] - 0s 176us/sample - loss: 0.3852 - acc: 0.9091
Epoch 907/1000
11/11 [==============================] - 0s 179us/sample - loss: 0.3851 - acc: 0.9091
Epoch

Epoch 980/1000
11/11 [==============================] - 0s 178us/sample - loss: 0.3765 - acc: 0.9091
Epoch 981/1000
11/11 [==============================] - 0s 191us/sample - loss: 0.3764 - acc: 0.9091
Epoch 982/1000
11/11 [==============================] - 0s 181us/sample - loss: 0.3762 - acc: 0.9091
Epoch 983/1000
11/11 [==============================] - 0s 194us/sample - loss: 0.3761 - acc: 0.9091
Epoch 984/1000
11/11 [==============================] - 0s 177us/sample - loss: 0.3760 - acc: 0.9091
Epoch 985/1000
11/11 [==============================] - 0s 183us/sample - loss: 0.3759 - acc: 0.9091
Epoch 986/1000
11/11 [==============================] - 0s 174us/sample - loss: 0.3758 - acc: 0.9091
Epoch 987/1000
11/11 [==============================] - 0s 171us/sample - loss: 0.3757 - acc: 0.9091
Epoch 988/1000
11/11 [==============================] - 0s 175us/sample - loss: 0.3756 - acc: 0.9091
Epoch 989/1000
11/11 [==============================] - 0s 193us/sample - loss: 0.3755 - ac

In [20]:
from sklearn.svm import SVC
SVM = SVC(gamma='auto')
H = SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
accurate_count = 0
test_count = 0
for label in y_test:
  if label == y_pred[test_count]:
     accurate_count += 1 
  test_count += 1
print("SVM Accuracy for all data: %.3f" % (accurate_count/test_count))

SVM Accuracy for all data: 0.833


/home/lab/anaconda3/envs/tf/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
H = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accurate_count = 0
test_count = 0
for label in y_test:
  if label == y_pred[test_count]:
     accurate_count += 1 
  test_count += 1
print("\nBayes Accuracy for all data: %.3f" % (accurate_count/test_count) ) 


Bayes Accuracy for all data: 0.833


/home/lab/anaconda3/envs/tf/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
block_data = np.empty([len(block_informations), 5], dtype = float)

for i in range(len(block_informations)):
  block_data[i] = [block_informations[i].pre_block,block_informations[i].post_block,block_informations[i].input,
                    block_informations[i].output,block_informations[i].memory]
  

block_labels = np.empty([len(block_informations), 1], dtype =int)
i = 0
with open(os.path.join(root,"experimentResults")) as sensivity_file:
  lines = sensivity_file.readlines()
  for line in lines:
    tempory = line.split(" ")
    name = tempory[0]+" "+tempory[1]
    if name not in block_names:
      continue
    if "N" in tempory[2]:
      block_labels[i] = int(0)
    else:
      block_labels[i] = int(1)
    i = i+1  
    
print(len(block_informations))
print(len(block_labels))
X_train, X_test, y_train, y_test = train_test_split(block_data,block_labels, test_size=0.33,shuffle=True)

23
23
